In [6]:
import pandas as pd 
import numpy as np


In [3]:
flights = pd.read_csv('flights-23-8-24-8.csv',
                      dtype={'Reporting_Airline': 'category', 
                             'Flight_Number_Reporting_Airline': 'uint16',
                             'Origin': 'category',
                             'Dest': 'category',
                             'DepDelay': 'float32',
                             'ArrDelay': 'float32',
                             'Cancelled': 'bool',
                             'CancellationCode': 'category'}, 
                      parse_dates=['DepTimestamp', 'ArrTimestamp'])

In [7]:
airports = np.unique(flights[['Origin', 'Dest']].values)

In [9]:
import csv
with open('airports.txt', 'w', newline='\n') as f: 
    writer = csv.writer(f, delimiter='\n')
    writer.writerow(airports)

The code below is a modified version of the IEM scraper example from IEM's ASOS scripts available on their Github. 

In [12]:
"""
Example script that scrapes data from the IEM ASOS download service
Source: https://github.com/akrherz/iem/blob/master/scripts/asos/iem_scraper_example.py
Utilizes this service: https://mesonet.agron.iastate.edu/request/download.phtml
"""
from __future__ import print_function
import json
import time
import datetime
# Python 2 and 3: alternative 4
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen

# Number of attempts to download data
MAX_ATTEMPTS = 6
# HTTPS here can be problematic for installs that don't have Lets Encrypt CA
SERVICE = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"


def download_data(uri):
    """Fetch the data from the IEM

    The IEM download service has some protections in place to keep the number
    of inbound requests in check.  This function implements an exponential
    backoff to keep individual downloads from erroring.

    Args:
      uri (string): URL to fetch

    Returns:
      string data
    """
    attempt = 0
    while attempt < MAX_ATTEMPTS:
        try:
            data = urlopen(uri, timeout=300).read().decode('utf-8')
            if data is not None and not data.startswith('ERROR'):
                return data
        except Exception as exp:
            print("download_data(%s) failed with %s" % (uri, exp))
            time.sleep(5)
        attempt += 1

    print("Exhausted attempts to download, returning empty data")
    return ""


def get_stations_from_filelist(filename):
    """Build a listing of stations from a simple file listing the stations.

    The file should simply have one station per line.
    """
    stations = []
    for line in open(filename):
        stations.append(line.strip())
    return stations


def get_stations_from_networks():
    """Build a station list by using a bunch of IEM networks."""
    stations = []
    states = """AK AL AR AZ CA CO CT DE FL GA HI IA ID IL IN KS KY LA MA MD ME
     MI MN MO MS MT NC ND NE NH NJ NM NV NY OH OK OR PA RI SC SD TN TX UT VA VT
     WA WI WV WY"""
    # IEM quirk to have Iowa AWOS sites in its own labeled network
    networks = ['AWOS']
    for state in states.split():
        networks.append("%s_ASOS" % (state,))

    for network in networks:
        # Get metadata
        uri = ("https://mesonet.agron.iastate.edu/"
               "geojson/network/%s.geojson") % (network,)
        data = urlopen(uri)
        jdict = json.load(data)
        for site in jdict['features']:
            stations.append(site['properties']['sid'])
    return stations


def main():
    """Our main method"""
    # timestamps in UTC to request data for
    startts = datetime.datetime(2023, 8, 1)
    endts = datetime.datetime(2024, 8, 31)

    service = SERVICE + "data=sknt&data=p01i&data=vsby&data=gust&data=skyc1&data=skyc2&data=skyc3&data=skyl1&data=skyl2&data=skyl3&data=wxcodes&tz=Etc/UTC&format=onlycomma&latlon=no&"

    service += startts.strftime('year1=%Y&month1=%m&day1=%d&')
    service += endts.strftime('year2=%Y&month2=%m&day2=%d&')

    # Two examples of how to specify a list of stations
    # stations = get_stations_from_networks()
    stations = get_stations_from_filelist("airports.txt")
    for station in stations:
        uri = '%s&station=%s' % (service, station)
        print('Downloading: %s' % (station, ))
        data = download_data(uri)
        outfn = 'weather/%s.csv' % (station)
        out = open(outfn, 'w')
        out.write(data)
        out.close()


if __name__ == '__main__':
    main()

Downloading: ABQ
Downloading: AMA
Downloading: ANC
Downloading: ASE
Downloading: ATL
Downloading: AUS
Downloading: BHM
Downloading: BNA
Downloading: BOS
Downloading: BRO
Downloading: BTR
Downloading: BWI
Downloading: BZN
Downloading: CHS
Downloading: CLE
Downloading: CLT
Downloading: CMH
Downloading: COS
Downloading: CRP
Downloading: CVG
Downloading: DAL
Downloading: DCA
Downloading: DEN
Downloading: DFW
Downloading: DSM
Downloading: DTW
Downloading: ECP
Downloading: EGE
Downloading: ELP
Downloading: EWR
Downloading: EYW
Downloading: FCA
Downloading: FLL
Downloading: GPT
Downloading: GRR
Downloading: GSP
Downloading: HDN
Downloading: HNL
Downloading: HOB
Downloading: HRL
Downloading: IAD
Downloading: IAH
Downloading: ICT
Downloading: IND
Downloading: JAC
Downloading: JAX
Downloading: JFK
Downloading: LAS
Downloading: LAX
Downloading: LBB
Downloading: LCH
Downloading: LFT
Downloading: LGA
Downloading: LIT
Downloading: LRD
Downloading: MAF
Downloading: MCI
Downloading: MCO
Downloading: M

Now, each flight's departure and arrival airport needs to have the correct weather data associated with it, based on the correspondingly scheduled departure and arrival times. 

In [32]:
import pandas as pd
import numpy as np 
import datetime as dt

# Load weather data with valid column as datetime index
airport_wx = {}
for airport in airports:
    ix = pd.read_csv(
        f'weather/{airport}.csv',
        parse_dates=['valid'],
        dtype={
            'station': 'category',
            'skyc1': 'category',
            'skyc2': 'category',
            'skyc3': 'category',
            'wxcodes': 'category'
        }
    )
    # Ensure valid is the datetime index
    ix.set_index('valid', inplace=True)
    airport_wx[airport] = ix


# For a given airport and timestamp, return cleaned and formatted weather data for the closest timestamp
def get_wx(airport, ts):
    if pd.isna(ts) or airport not in airport_wx:
        # Return default values for missing timestamps or invalid airports
        return pd.DataFrame({'ceiling': [25000], 'wind': [0], 'visibility': [10], 'precip': [0]})

    airport_wxdf = airport_wx[airport]
    #print(f"Flight Timestamp: {ts}")
    #print(f"Weather Data Timestamps: {airport_wxdf.index[:5]}")


    # Find the closest timestamp using numpy's abs
    try:
        deltas = np.abs(airport_wxdf.index - ts)
        closest_idx = deltas.argmin()  # Get the index of the smallest difference
        airport_wx_data = airport_wxdf.iloc[closest_idx]
    except Exception as e:
        print(f"Error finding closest timestamp for {airport} at {ts}: {e}")
        # Return default values if matching fails
        return pd.DataFrame({'ceiling': [25000], 'wind': [0], 'visibility': [10], 'precip': [0]})

    # Process weather data
    ceiling = 25000
    if airport_wx_data['skyc1'] in ['OVC', 'BKN']:
        ceiling = pd.to_numeric(airport_wx_data['skyl1'], errors='coerce', downcast='unsigned')
    elif airport_wx_data['skyc2'] in ['OVC', 'BKN']:
        ceiling = pd.to_numeric(airport_wx_data['skyl2'], errors='coerce', downcast='unsigned')
    elif airport_wx_data['skyc3'] in ['OVC', 'BKN']:
        ceiling = pd.to_numeric(airport_wx_data['skyl3'], errors='coerce', downcast='unsigned')

    wind = pd.to_numeric(airport_wx_data.get('sknt', 0), errors='coerce', downcast='unsigned')
    visibility = pd.to_numeric(airport_wx_data.get('vsby', 10), errors='coerce', downcast='unsigned')
    precip = pd.to_numeric(airport_wx_data.get('p01i', 0), errors='coerce', downcast='unsigned')

    return pd.DataFrame({'ceiling': [ceiling], 'wind': [wind], 'visibility': [visibility], 'precip': [precip]})

# For a single flight, return a Series containing the origin and destination weather
def get_wx_for_flights(df):
    origin_airport = df['Origin']
    scheduled_departure = df['DepTimestamp']
    destination_airport = df['Dest']
    scheduled_arrival = df['ArrTimestamp']

    # Handle missing timestamps
    origin_wx = get_wx(origin_airport, scheduled_departure)
    destination_wx = get_wx(destination_airport, scheduled_arrival)
    
    #print("Origin Weather:", origin_wx)
    #print("Destination Weather:", destination_wx)
    
    origin_wx = origin_wx.add_prefix('ORIGIN_').iloc[0]
    destination_wx = destination_wx.add_prefix('DEST_').iloc[0]
    return pd.concat([origin_wx, destination_wx])


# Apply weather data processing to each row
wx_data = flights.apply(lambda x: get_wx_for_flights(x), axis=1)

print(wx_data.head())
print(wx_data.iloc[0])  # Inspect the first row's weather data


# Combine all the DataFrames in wx_data into a single DataFrame
#wx_df = pd.concat([pd.DataFrame([row]) for row in wx_data], ignore_index=True)
#print(wx_df.head())
#print(wx_df.iloc[0])  # Inspect the first row's weather data

# Concatenate the flight and weather DataFrames
flights.reset_index(drop=True, inplace=True)
flights_with_wx = pd.concat([flights, wx_data], axis=1)
flights_with_wx.to_csv('flights-wx.csv', index=False)
print("Data saved to 'flights-wx.csv'")




C:\Users\ams21\AppData\Local\Temp\ipykernel_6624\3340997814.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  ix = pd.read_csv(
C:\Users\ams21\AppData\Local\Temp\ipykernel_6624\3340997814.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  ix = pd.read_csv(
C:\Users\ams21\AppData\Local\Temp\ipykernel_6624\3340997814.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  ix = pd.read_csv(
C:\Users\ams21\AppData\Local\Temp\ipykernel_6624\3340997814.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  ix = pd.read_csv(
C:\Users\ams21\AppData\Local\Temp\ipykernel_6624\3340997814.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  ix = pd.read_csv(


Error finding closest timestamp for SJU at 2023-08-31 00:10:00: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
Error finding closest timestamp for ANC at 2023-08-31 19:33:00: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
Error finding closest timestamp for SJU at 2023-08-31 15:17:00: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
Error finding closest timestamp for HNL at 2023-08-31 13:20:00: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
Error finding closest timestamp for SJU at 2023-08-30 02:34:00: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
Error finding closest timestamp for ANC at 2023-08-30 19:09:00: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
Error finding closest timestamp for SJU at 2023-08-30 15:46:00: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
Error finding closest timestamp for HNL at 2023-08-30 13:20:00: unsupported operand